# モデル評価と選択の演習




In [ ]:
import pandas as pd
import numpy as  np
import csv
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import confusion_matrix,  accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot  as plt
%matplotlib inline

In [ ]:
# Colaboratoryでは以下を実行して必要なファイルをダウンロード
!wget https://raw.githubusercontent.com/UTDataMining/2021S/master/ex10/winequality-red.csv

UCI Machine Learning Repositoryに公開されているワインの品質データセットを用いて、ワインの理化学検査結果に基づく特徴量からワインの品質を予測することを考えます。

以下では赤ワインのデータセットを使用します。

[Wine Quality Data Set ](https://archive.ics.uci.edu/ml/datasets/wine+quality)

各ワインのデータは以下の11種類の特徴量からなります。
- 1 - fixed acidity 
- 2 - volatile acidity 
- 3 - citric acid 
- 4 - residual sugar 
- 5 - chlorides 
- 6 - free sulfur dioxide 
- 7 - total sulfur dioxide 
- 8 - density 
- 9 - pH 
- 10 - sulphates 
- 11 - alcohol 

また、各ワインには0から10の品質スコアが付与さています。11種類の特徴量を元にワインの品質を高品質（ワインのqualityが6以上）、低品質（ワインのqualityが6未満）の2値にして予測する2クラス分類問題を考えます。

In [ ]:
df = pd.read_csv("winequality-red.csv", sep=";")
df.head()

In [ ]:
# 特徴量
X=df[ ['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates','alcohol']].values

# 正解ラベルを1（高品質：ワインのqualityが6以上）か0（低品質：ワインのqualityが6未満）とする
y=df['quality'].values
y = (y>=6).astype(np.int) 
print(np.sum(y==1, axis=0)) # 高品質ワインデータの数
print(np.sum(y==0, axis=0)) # 低品質ワインデータの数

In [ ]:
# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8, stratify=y) 

まず、すべての特徴量を使ってロジスティック回帰によりワイン品質の分類をしてみます

[LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

In [ ]:
# 標準化
sc = StandardScaler()
sc.fit(X_train)
norm_X_train =  sc.transform(X_train)
norm_X_test =  sc.transform(X_test)

# ロジスティック回帰モデル
lr=LogisticRegression(solver='liblinear',  multi_class='auto') 

lr.fit(norm_X_train, y_train)    
y_pred = lr.predict(norm_X_test)

print(confusion_matrix(y_test, y_pred)) # 混同行列
print(accuracy_score(y_test, y_pred)) # accuracy
print(precision_score(y_test, y_pred)) # precision
print(recall_score(y_test, y_pred)) # recall
print(f1_score(y_test, y_pred))  # f値

# 特徴量ごとのパラメータ重み
pd.DataFrame(lr.coef_, index=['weight'], columns=df.drop("quality", axis=1).columns.values)

## Q1
先のモデルについて、訓練データを用いた**5分割**交差検証により**F値**を評価指標として最適な正則化項の係数を決定する以下の`model_selection1`関数を作成してください。`model_selection1`関数は訓練データの入力、訓練データの出力をそれぞれ引数に受け取り、交差検証に基づいてハイパーパラメータ`C`の最適な値（`C`の値の逆数が正則化項の係数）を返します。

- `C`は、 `0.01`, `0.1`, `1`, `10`, `100`のいずれかとします。

In [ ]:
def model_selection1(X_train, y_train):
    pipe = make_pipeline(StandardScaler(), LogisticRegression(solver='liblinear', multi_class='auto'))
    param_grid = {'logisticregression__C': [0.01, 0.1, 1, 10, 100]}
    grid = GridSearchCV(..., param_grid=..., scoring="...", cv=...)
    grid.fit(..., ...)
    return grid.best_params_['logisticregression__C'], grid.best_score_

関数が完成したら以下のセルを実行して動作を確認してください。交差検証の結果、最適な`C`の値は10、その時のF値は0.7614...となります。

In [ ]:
model_selection1(X_train, y_train)

## Q2
Q1のモデルについて、さらに多項式を用いた特徴量生成によりモデルを複雑化することを考えます。訓練データを用いた**5分割**交差検証により**F値**を評価指標として最適な正則化項の係数と多項式の次数の組み合わせ決定する以下の`model_selection2`関数を作成してください。`model_selection2`関数は訓練データの入力、訓練データの出力をそれぞれ引数に受け取り、交差検証に基づいてハイパーパラメータ`C`と次数`degree`の最適な値を返します。

- `C`は、 `0.01`, `0.1`, `1`, `10`, `100`のいずれかとします。
- 多項式の次数`degree`は、`1`, `2`, `3`のいずれかとします。


In [ ]:
def model_selection2(X_train, y_train):
    pipe = make_pipeline(PolynomialFeatures(), StandardScaler(), LogisticRegression(solver='liblinear', multi_class='auto'))
    param_grid = {'logisticregression__C': ..., 'polynomialfeatures__degree': ...}
    grid = GridSearchCV(..., param_grid=..., scoring="...", cv=...)
    grid.fit(..., ...)
    return grid.best_params_['logisticregression__C'], grid.best_params_['polynomialfeatures__degree'], grid.best_score_

関数が完成したら以下のセルを実行して動作を確認してください（少し時間がかかります）。交差検証の結果、最適な`C`の値は0.1、`degree`の値は3、その時のF値は0.7664...となります。

In [ ]:
optimal_C, optimal_degree, valid_f1 = model_selection2(X_train, y_train)
print(optimal_C, optimal_degree, valid_f1)

Q2の交差検証で決定した正則化項の係数と多項式の次数により訓練データからロジスティック回帰モデルを学習し、テストデータで評価を行ってみます。モデル選択をする前と比べた予測精度の変化を確認してください。

In [ ]:
pipe = make_pipeline(PolynomialFeatures(degree=optimal_degree), StandardScaler(), LogisticRegression(solver='liblinear',  multi_class='auto', C=optimal_C))
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)

print(confusion_matrix(y_test, y_pred)) # 混同行列
print(accuracy_score(y_test, y_pred)) # accuracy
print(precision_score(y_test, y_pred)) # precision
print(recall_score(y_test, y_pred)) # recall
print(f1_score(y_test, y_pred))  # f値

## Q3
上記のロジスティック回帰モデルによるワイン品質の分類問題について、独自に特徴量の前処理、変換、選択、作成などを行い、その時のテストデータの精度を確認してください。

In [ ]:
X=df[ ['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates','alcohol']].values
y=df['quality'].values
y = (y>=6).astype(np.int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8, stratify=y)

...
X_train = # your own X_train here 
X_test = # your own X_test here

lr = LogisticRegression(...) 
lr.fit(X_train, y_train)     
y_pred = lr.predict(X_test)

print(confusion_matrix(y_test, y_pred)) # 混同行列
print(accuracy_score(y_test, y_pred)) # accuracy
print(precision_score(y_test, y_pred)) # precision
print(recall_score(y_test, y_pred)) # recall
print(f1_score(y_test, y_pred))  # f値

## コードのテスト

In [ ]:
## コードのテストの前にこのセルを実行してください
!pip install prog_edu_assistant_tools
import re
import sys
import jinja2
from IPython.core import display
from google.colab import _message as google_message
from prog_edu_assistant_tools.magics import report, autotest, CaptureOutput
from prog_edu_assistant_tools.check import Check

## Q1

In [ ]:
# Run this cell to check your solution.
# If you get an error 'Check not defined', make sure you have run all preceding
# cells once (Runtime -> Run before)
X=df[ ['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates','alcohol']].values
y=df['quality'].values
y = (y>=6).astype(np.int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8, stratify=y) 
Check('q1')

## Q2
少し時間がかかります

In [ ]:
# Run this cell to check your solution.
# If you get an error 'Check not defined', make sure you have run all preceding
# cells once (Runtime -> Run before)
X=df[ ['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates','alcohol']].values
y=df['quality'].values
y = (y>=6).astype(np.int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8, stratify=y)
Check('q2')